Extracting Data From API into bronze layer

In [0]:
tiers=['bronze','silver','gold']
adls_paths = {tier: f"abfss://{tier}@earthquakedatafiles.dfs.core.windows.net/" for tier in tiers}

# Accessing paths
bronze_adls = adls_paths["bronze"]
silver_adls = adls_paths["silver"]
gold_adls = adls_paths["gold"] 

dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)

[]

In [0]:
import requests
import json
from datetime import date,timedelta

In [0]:
start_date=date.today() - timedelta(1)
end_date=date.today()

###Fetching data from api

url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

try:
    # Make the GET request to fetch data
    response = requests.get(url)

    # Check if the request was successful
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    data = response.json().get('features', [])

    if not data:
        print("No data returned for the specified date range.")
    else:
        # Specify the ADLS path
        file_path = f"{bronze_adls}/{start_date}_earthquake_data.json"

        # Save the JSON data
        json_data = json.dumps(data, indent=4)
        dbutils.fs.put(file_path, json_data, overwrite=True)
        print(f"Data successfully saved to {file_path}")
except requests.exceptions.RequestException as e:
    print(f"Error fetching data from API: {e}")



Wrote 333832 bytes.
Data successfully saved to abfss://bronze@earthquakedatafiles.dfs.core.windows.net//2025-07-19_earthquake_data.json


In [0]:
##having a quick glimpse of the data
data[0:5]

[{'type': 'Feature',
  'properties': {'mag': 0.91,
   'place': '13 km NE of Coso Junction, CA',
   'time': 1752969268010,
   'updated': 1752969476770,
   'tz': None,
   'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ci41026639',
   'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ci41026639&format=geojson',
   'felt': None,
   'cdi': None,
   'mmi': None,
   'alert': None,
   'status': 'automatic',
   'tsunami': 0,
   'sig': 13,
   'net': 'ci',
   'code': '41026639',
   'ids': ',ci41026639,',
   'sources': ',ci,',
   'types': ',nearby-cities,origin,phase-data,scitech-link,',
   'nst': 21,
   'dmin': 0.01561,
   'rms': 0.14,
   'gap': 56,
   'magType': 'ml',
   'type': 'earthquake',
   'title': 'M 0.9 - 13 km NE of Coso Junction, CA'},
  'geometry': {'type': 'Point',
   'coordinates': [-117.8531667, 36.1331667, 6.55]},
  'id': 'ci41026639'},
 {'type': 'Feature',
  'properties': {'mag': 1.6,
   'place': '37 km E of Pedro Bay, Alaska',
   'time': 175296851602

In [0]:
output_data={
    "start_date":start_date.isoformat(),
    "end_date":end_date.isoformat(),
    "bronze_adls":bronze_adls,
    "silver_adls":silver_adls,
    "gold_adls":gold_adls
}
dbutils.jobs.taskValues.set(key="bronze_output",value=output_data)
print (output_data)

{'start_date': '2025-07-19', 'end_date': '2025-07-20', 'bronze_adls': 'abfss://bronze@earthquakedatafiles.dfs.core.windows.net/', 'silver_adls': 'abfss://silver@earthquakedatafiles.dfs.core.windows.net/', 'gold_adls': 'abfss://gold@earthquakedatafiles.dfs.core.windows.net/'}
